In [13]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
dataset_path = '/content/drive/MyDrive/Imagedata/Imagedata'

In [15]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (224, 224))  # Resize images to 224x224 pixels
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")
    images = np.array(images, dtype='float32') / 255.0  # Normalize images
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))
    return images, labels, label_names

images, labels, label_names = load_dataset(dataset_path)


100%|██████████| 22/22 [00:34<00:00,  1.58s/it]


In [17]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [18]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/20
8/8 [==============================] - 70s 7s/step - loss: 3.6886 - accuracy: 0.0508 - val_loss: 3.0446 - val_accuracy: 0.1562
Epoch 2/20
8/8 [==============================] - 59s 7s/step - loss: 2.8905 - accuracy: 0.1836 - val_loss: 2.9359 - val_accuracy: 0.1250
Epoch 3/20
8/8 [==============================] - 59s 7s/step - loss: 2.5166 - accuracy: 0.3047 - val_loss: 2.8145 - val_accuracy: 0.1719
Epoch 4/20
8/8 [==============================] - 59s 7s/step - loss: 2.1194 - accuracy: 0.4492 - val_loss: 2.7126 - val_accuracy: 0.1875
Epoch 5/20
8/8 [==============================] - 59s 7s/step - loss: 1.5673 - accuracy: 0.6562 - val_loss: 2.6283 - val_accuracy: 0.2500
Epoch 6/20
8/8 [==============================] - 58s 7s/step - loss: 1.0629 - accuracy: 0.7656 - val_loss: 2.5633 - val_accuracy: 0.2031
Epoch 7/20
8/8 [==============================] - 59s 8s/step - loss: 0.6224 - accuracy: 0.9102 - val_loss: 2.4843 - val_accuracy: 0.3125
Epoch 8/20
8/8 [==================

In [23]:
import cv2
import numpy as np

def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:
            break

        # Preprocess the frame for prediction (resize, expand dimensions, normalize)
        face = cv2.resize(frame, (224, 224))  # Assuming your model expects 224x224 inputs
        face = np.expand_dims(face, axis=0)  # Add batch dimension
        face = face / 255.0  # Normalize pixel values to [0, 1] as during training

        prediction = model.predict(face)  # Make prediction
        predicted_class = label_names[np.argmax(prediction)]  # Get the name of the predicted class

        # Display the predicted class on the frame
        cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Real-time Face Recognition', frame)

        # Break the loop when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
# real_time_recognition(model, label_names)
